# Customer Churn Prediction with XGBoost
<p>Using Gradient Boosted Trees to Predict Mobile Customer Departure</p>

## Background

_This notebook has been adapted from an [AWS blog post](https://aws.amazon.com/blogs/ai/predicting-customer-churn-with-amazon-machine-learning/)_

Losing customers is costly for any business.  Identifying unhappy customers early on gives you a chance to offer them incentives to stay.  This notebook describes using machine learning (ML) for the automated identification of unhappy customers, also known as customer churn prediction.

## Setup

### Imports

In [ ]:
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker import get_execution_role
import pandas as pd
import sagemaker
import boto3
import re
import os

In [ ]:
sagemaker.__version__

### S3 Bucket + Session + Role

In [ ]:
session = sagemaker.Session()

bucket = session.default_bucket()
prefix = 'xgboost-churn'

role = get_execution_role()
role

### Container

In [ ]:
container_image_uri = get_image_uri(boto3.Session().region_name, 'xgboost', '1.0-1')
container_image_uri

### S3 Pointers 

In [ ]:
s3_train_data_path = 's3://{}/{}/train'.format(bucket, prefix)
s3_train_pointer = sagemaker.s3_input(s3_data=s3_train_data_path, content_type='csv')
s3_validation_data_path = 's3://{}/{}/validation'.format(bucket, prefix)
s3_validation_pointer = sagemaker.s3_input(s3_data=s3_validation_data_path, content_type='csv')
s3_test_data_path = 's3://{}/{}/test'.format(bucket, prefix)
s3_test_pointer = sagemaker.s3_input(s3_data=s3_test_data_path, content_type='csv')

In [ ]:
s3_train_data_path

In [ ]:
s3_train_pointer.__dict__

### Load Data from Local to S3

By modern standards, it’s a relatively small dataset, with only 3,333 records, where each record uses 21 attributes to describe the profile of a customer of an unknown US mobile operator. The attributes are:

- `State`: the US state in which the customer resides, indicated by a two-letter abbreviation; for example, OH or NJ
- `Account Length`: the number of days that this account has been active
- `Area Code`: the three-digit area code of the corresponding customer’s phone number
- `Phone`: the remaining seven-digit phone number
- `Int’l Plan`: whether the customer has an international calling plan: yes/no
- `VMail Plan`: whether the customer has a voice mail feature: yes/no
- `VMail Message`: presumably the average number of voice mail messages per month
- `Day Mins`: the total number of calling minutes used during the day
- `Day Calls`: the total number of calls placed during the day
- `Day Charge`: the billed cost of daytime calls
- `Eve Mins, Eve Calls, Eve Charge`: the billed cost for calls placed during the evening
- `Night Mins`, `Night Calls`, `Night Charge`: the billed cost for calls placed during nighttime
- `Intl Mins`, `Intl Calls`, `Intl Charge`: the billed cost for international calls
- `CustServ Calls`: the number of calls placed to Customer Service
- `Churn?`: whether the customer left the service: true/false

The last attribute, `Churn?`, is known as the target attribute–the attribute that we want the ML model to predict.  Because the target attribute is binary, our model will be performing binary prediction, also known as binary classification.

Let's begin exploring the data:

In [ ]:
churn = pd.read_csv('./DATA/churn.txt')
pd.set_option('display.max_columns', 500)
churn

<b> Load & Inspect Train & Validation Sets (Local) </b>

In [ ]:
train = pd.read_csv('./DATA/train.csv', sep=',')
train.head()

In [ ]:
train.shape 

In [ ]:
validation = pd.read_csv('./DATA/validation.csv', sep=',')
validation.head()

In [ ]:
validation.shape 

In [ ]:
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/train.csv')).upload_file('./DATA/train.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation/validation.csv')).upload_file('./DATA/validation.csv')

<b> Ensure we can read the data from S3 </b>

In [ ]:
train_df = pd.read_csv('/'.join([s3_train_data_path, 'train.csv']), sep=',')
train_df.head()

## Train

Now, we can specify a few parameters like what type of training instances we'd like to use and how many, as well as our XGBoost hyperparameters.  A few key hyperparameters are:
- `max_depth` controls how deep each tree within the algorithm can be built.  Deeper trees can lead to better fit, but are more computationally expensive and can lead to overfitting.  There is typically some trade-off in model performance that needs to be explored between a large number of shallow trees and a smaller number of deeper trees.
- `subsample` controls sampling of the training data.  This technique can help reduce overfitting, but setting it too low can also starve the model of data.
- `num_round` controls the number of boosting rounds.  This is essentially the subsequent models that are trained using the residuals of previous iterations.  Again, more rounds should produce a better fit on the training data, but can be computationally expensive or lead to overfitting.
- `eta` controls how aggressive each round of boosting is.  Larger values lead to more conservative boosting.
- `gamma` controls how aggressively trees are grown.  Larger values lead to more conservative models.

More detail on XGBoost's hyperparmeters can be found on their GitHub [page](https://github.com/dmlc/xgboost/blob/master/doc/parameter.md).

In [ ]:
xgb = sagemaker.estimator.Estimator(container_image_uri,
                                    role, 
                                    train_instance_count=1, 
                                    train_instance_type='ml.m5.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=session)

xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        num_round=100)

xgb.fit({'train': s3_train_pointer, 'validation': s3_validation_pointer}) 

## Host

Now that we've trained a model, let's deploy it to a hosted endpoint.

In [ ]:
xgb_predictor = xgb.deploy(initial_instance_count = 1, instance_type = 'ml.m5.xlarge')

In [ ]:
xgb_predictor.endpoint

## Evaluate

Now that we have a hosted endpoint running, we can make real-time predictions from our model very easily, simply by making an http POST request.  But first, we'll need to setup serializers and deserializers for passing our `test_data` NumPy arrays to the model behind the endpoint.

In [ ]:
from sagemaker.predictor import csv_serializer

In [ ]:
xgb_predictor.content_type = 'text/csv'
xgb_predictor.serializer = csv_serializer

Now, we'll use a simple function to:
1. Loop over our test dataset
1. Split it into mini-batches of rows 
1. Convert those mini-batchs to CSV string payloads
1. Retrieve mini-batch predictions by invoking the XGBoost endpoint
1. Collect predictions and convert from the CSV output our model provides into a NumPy array

In [ ]:
def predict(data, rows=500):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, xgb_predictor.predict(array).decode('utf-8')])

    return np.fromstring(predictions[1:], sep=',')

In [ ]:
import pandas as pd

test_df = pd.read_csv('./DATA/test.csv', sep=',')
test_df.head()

In [ ]:
predictions = predict(test_df.to_numpy()[:, 1:])

In [ ]:
predictions

In [ ]:
THRESHOLD = 0.3

In [ ]:
pd.crosstab(index=test_df.iloc[:, 0], columns=np.where(predictions > THRESHOLD, 1, 0))